# Cours de Python - IIIF et Manipulation de Fichiers Historiques 1

---

## Table des Matières

1. [Révisions](#Révisions)
    - [Héritage](#Héritage)
        - [Qu'est-ce que l'héritage en POO ?](#Qu'est-ce-que-l'héritage-en-POO-?)
        - [Utilisation de `super()`](#Utilisation-de-super--)
        - [Exemple pratique](#Exemple-pratique)
    - [Les paramètres `*args` et `**kwargs`](#Les-paramètres-args-et-kwargs)
        - [Utilisation de `*args`](#Utilisation-de-args)
        - [Utilisation de `**kwargs`](#Utilisation-de-kwargs)
        - [Combinaison de `*args` et `**kwargs`](#Combinaison-de-args-et-kwargs)
2. [Introduction à IIIF](#Introduction-à-IIIF)
    - [Qu'est-ce que IIIF ?](#Qu'est-ce-que-IIIF-?)
    - [Les API IIIF](#Les-API-IIIF)
3. [Structure des Manifestes IIIF](#Structure-des-Manifestes-IIIF)
    - [Le format JSON-LD](#Le-format-JSON-LD)
    - [Composants d'un manifeste IIIF](#Composants-d'un-manifeste-IIIF)
4. [La bibliothèque `json`](#La-bibliothèque-json)
    - [Chargement de données JSON](#Chargement-de-données-JSON)
    - [Écriture de données JSON](#Écriture-de-données-JSON)
5. [Parcourir un manifeste IIIF](#Parcourir-un-manifeste-IIIF)
    - [Chargement du manifeste](#Chargement-du-manifeste)
    - [Extraction des métadonnées historiques](#Extraction-des-métadonnées-historiques)
6. [Manipuler des images avec `PIL`](#Manipuler-des-images-avec-PIL)
    - [Chargement et affichage d'images historiques](#Chargement-et-affichage-d'images-historiques)
    - [Transformation d'images](#Transformation-d'images)
7. [Gérer le téléchargement avec `pathlib` et `requests`](#gérer-le-téléchargement-avec-pathlib)
    - [Création de répertoires pour les documents historiques](#Création-de-répertoires-pour-les-documents-historiques)
    - [Téléchargement d'images à partir de manifestes IIIF](#Téléchargement-d'images-à-partir-de-manifestes-IIIF)
8. [Téléchargement multiple d’images](#Téléchargement-multiple-d’images)
    - [Boucle de téléchargement](#Boucle-de-téléchargement)
    - [Améliorations possibles](#Améliorations-possibles)

    ----

## Révisions

### Héritage

#### Qu'est-ce que l'héritage en POO ?

L'héritage est un mécanisme en programmation orientée objet (POO) qui permet à une classe (appelée **classe enfant** ou **sous-classe**) d'hériter des attributs et méthodes d'une autre classe (appelée **classe parente** ou **super-classe**).

- **But** : Réutiliser du code existant, établir une hiérarchie entre les classes pour une meilleure organisation.

#### Utilisation de `super()`

La fonction `super()` est utilisée pour appeler des méthodes ou des constructeurs de la classe parente.

- **Avantages** :
  - Assure une initialisation correcte des attributs hérités.
  - Permet de réutiliser le code de la classe parente.

#### Exemple pratique

Imaginons que nous travaillons sur des documents historiques et que nous voulons modéliser des types de documents.

In [ ]:
class DocumentHistorique:
    def __init__(self, titre, date):
        self.titre = titre
        self.date = date

    def afficher_info(self):
        print(f"Titre : {self.titre}")
        print(f"Date : {self.date}")

# Classe enfant
class Manuscrit(DocumentHistorique):
    def __init__(self, titre, date, auteur):
        super().__init__(titre, date)
        self.auteur = auteur

    def afficher_info(self):
        super().afficher_info()
        print(f"Auteur : {self.auteur}")

In [ ]:
manuscrit = Manuscrit("Bestiaire divin", "XIIIe siècle", "Guillaume le Clerc")
manuscrit.afficher_info()

#### Exercice

Créez une classe `Lettre` qui hérite de `DocumentHistorique` qui ajoute un attribut `destinataire` et modifie la méthode `afficher_info`

In [ ]:
# Votre code ici

---

### Les paramètres `*args` et `**kwargs`

#### Utilisation de `*args`

- **`*args`** permet de passer un nombre variable d'arguments positionnels à une fonction.
- Les arguments sont accessibles sous forme de **tuple**.

In [ ]:
def lister_evenements(*evenements):
    print("Liste des événements historiques :")
    for evenement in evenements:
        print(f"- {evenement}")

# Utilisation
lister_evenements("Fondation de l'Empire akkadien", "Civilisation minoenne", "Chute de Rome", "Règne de saint Louis")

#### Utilisation de `**kwargs`

- **`**kwargs`** permet de passer un nombre variable d'arguments nommés à une fonction.
- Les arguments sont accessibles sous forme de **dictionnaire**.

In [ ]:
def afficher_informations_document(**infos):
    print("Informations sur le document :")
    for cle, valeur in infos.items():
        print(f"{cle} : {valeur}")

# Utilisation
afficher_informations_document(titre="De animalibus", auteur= 'Albertus Magnus', date="1270", lieu="Cologne")

#### Combinaison de `*args` et `**kwargs`

- Les deux paramètres peuvent être passés simultanment à une fonction. 
- **A noter** : `args` et `kwargs` sont des convebtions de nommage vous pouvez passer le nom des arguments que vous souhaitez en respectant `*` pour les arguments positionnels et `**` pour les arguments nommés.

In [ ]:
def fonction_mixte(*args, **kwargs):
    print("Arguments positionnels :", args)
    print("Arguments nommés :", kwargs)

# Utilisation
fonction_mixte("Renaissance", "Moyen Âge", periode1="Antiquité", periode2="Époque moderne")

---

## Introduction à IIIF

### Qu'est-ce que IIIF ?

- **IIIF** : International Image Interoperability Framework
- **Objectif** : Fournir un cadre d'interopérabilité pour diffuser, présenter et annoter des images et documents historiques sur le Web.

### Les API IIIF

- **API Présentation** : Décrit la structure, les métadonnées et les liens d'une ressource numérique.
- **API Image** : Permet d'accéder et de manipuler des images en haute résolution via une syntaxe d'URL standard.

---

## Structure des Manifestes IIIF

### Le format JSON-LD

- **JSON-LD** : Extension de JSON pour intégrer des données liées (Linked Data).
- Permet d'ajouter du contexte aux données pour une meilleure interopérabilité.

### Composants d'un manifeste IIIF

- **`@context`** : Définit le contexte JSON-LD.
- **`@id`** : Identifiant unique de la ressource.
- **`@type`** : Type de la ressource (e.g., `sc:Manifest`).
- **`label`** : Titre ou nom de la ressource.
- **`metadata`** : Liste de métadonnées supplémentaires.
- **`items`** ou **`sequences`** : Liste des éléments structurants (pages, images).

---

## La bibliothèque `json`

### Chargement de données JSON

La bibliothèque `json` est essentielle pour manipuler des données au format JSON, comme les manifestes IIIF.

In [ ]:
import json

# Charger un fichier JSON local
with open('document_historique.json', 'r', encoding='utf-8') as fichier:
    data = json.load(fichier)

# Charger des données JSON depuis une chaîne
json_str = '{"titre": "Déclaration d\'indépendance des États-Unis", "date": "1776"}'
data = json.loads(json_str)
print(data)

### Écriture de données JSON

In [ ]:
import json

# Écrire des données dans un fichier JSON
document = {
    'titre': 'Constitution française',
    'date': '1791',
    'auteurs': ['Assemblée nationale']
}
with open('constitution_1791.json', 'w', encoding='utf-8') as fichier:
    json.dump(document, fichier, indent=4, ensure_ascii=False)

- **Note** : L'option `ensure_ascii=False` permet de conserver les caractères spéciaux (accents) lors de l'écriture.

---

## Parcourir un manifeste IIIF

### Chargement du manifeste

Nous allons travailler avec le manifeste de **Gallica** :

In [ ]:
import json

def open_json(json_file):
        with open(json_file, 'r') as readable_json:
                manifeste = json.load(readable_json)
        return manifeste

### Extraction des métadonnées historiques

Deux manières de parcourir un manifeste : 
- Comme un dictionaire en appelant les clés et les valeurs (Exemple 1)
- En parcourant simplement les clés (Exemple 1) : plus fluide pour récupérer les données qui nous intéresse et parourir la structure

In [ ]:
iiif_manifest = open_json('pathtomanifeste')

# Exemple 1
for key, value in iiif_manifest.items():
    print(key)

# Exemple 2
for item in iiif_manifest:
    print(item)

# Avantage de la deuxième pour taper sur les clés qui nous intéresse
# id = iiif_manifest['@id']

#### Exercice

- Chargez l'un des manifestes suivants :

  - **BVMM** : `https://bvmm.irht.cnrs.fr/iiif/22470/manifest`
  - **e-codices** : `https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/manifest.json`

- Affichez le titre et les métadonnées du manifeste.

In [ ]:
# Votre code ici

---

## Manipuler des images avec `PIL`

### Chargement et affichage d'images historiques

Nous allons manipuler des images issus de l'un des manifestes précédents

In [ ]:
from PIL import Image

# Ouvrir une image locale
image = Image.open('pathto/img.jpg')

# Afficher l'image
image.show()

### Transformation d'images

Par exemple, nous pouvons améliorer la lisibilité d'un manuscrit ancien.

In [ ]:
# Convertir en niveaux de gris
image_gris = image.convert('L')

# Augmenter le contraste
from PIL import ImageEnhance

enhancer = ImageEnhance.Contrast(image_gris)
image_contraste = enhancer.enhance(2)  # Augmenter le contraste par un facteur de 2

# Enregistrer l'image modifiée
image_contraste.save('images_historique/page1_contraste.jpg')

# Afficher l'image modifiée
image_contraste.show()

#### Exercice interactif

- Chargez une image  de votre choix.
- Effectuez les opérations suivantes :
  - Redimensionnez l'image pour qu'elle ait une largeur de 800 pixels tout en conservant le ratio.
  - Appliquez un filtre de netteté pour améliorer la lisibilité (parcourir la doc : https://pillow.readthedocs.io/en/stable/reference/ImageFilter.html).

In [ ]:
# Votre code ici

---

## Gérer le téléchargement avec `pathlib` et `requests`



### Gérer le téléchargement avec `pathlib`

Le module `pathlib` permet de **manipuler les fichiers et répertoires** sous forme d’**objets Python**, offrant une approche plus **lisible**, **sûre** et **portable** que `os`.

### Fonctions utiles

- **Vérifier si un chemin existe** : `p.exists()`  
- **Créer un répertoire** : `p.mkdir(parents=True, exist_ok=True)`  
- **Joindre des chemins** : `p = dossier / "fichier.txt"`

### Exemple

```python
from pathlib import Path

# Chemin du répertoire où enregistrer les images
repertoire_images = Path("images")

# Créer le répertoire s'il n'existe pas
repertoire_images.mkdir(parents=True, exist_ok=True)
```

#### Exemple :

```python
from pathlib import Path

# Chemin du répertoire où enregistrer les images
repertoire_images = Path("images")

# Créer le répertoire s'il n'existe pas
repertoire_images.mkdir(parents=True, exist_ok=True)
```

`Path` gère automatiquement la syntaxe des chemins, quel que soit le système d’exploitation
(Windows, macOS, Linux), et permet d’écrire un code **plus clair** et **maintenable**
pour le téléchargement et la gestion des fichiers.

### Création de répertoires pour les documents historiques

In [2]:
from pathlib import Path

# Chemin du répertoire
dossier_images = Path('pathtofolder')

# Créer le répertoire s'il n'existe pas
if not dossier_images.exists():
    dossier_images.mkdir(parents=True)
    print(f"Dossier '{dossier_images}' créé.")
else:
    print(f"Dossier '{dossier_images}' existe déjà.")

Dossier 'pathtofolder' créé.


### Téléchargement d'images à partir de manifestes IIIF

Nous allons télécharger une image à partir du manifeste chargé précédemment, en suivant les étapes :
- Ouvrir le manifeste en ligne
- Parcourir la structure pour trouver l'url de la première image
- L'enregistrer dans un dossier spécifique avec comme nom `image_(numéro canvas + 1)`

In [ ]:
import requests
import time
from pathlib import Path

def download_image_from_url(image_url: str, dir_path: Path, request_pause: int) -> None:
    """
    Télécharge une image à partir d'une URL et l'enregistre sous forme de fichier.

    Paramètres :
    - image_url (str) : URL de l'image à télécharger.
    - dir_path (Path) : Chemin du dossier où l'image sera enregistrée.
    - request_pause (int) : Durée de la pause en secondes après la requête.
    """
    
    print(f"Téléchargement de l'image depuis {image_url}")

    try:
        # Requête pour télécharger l'image
        r = requests.get(image_url, stream=True)
        print(f"Code de statut : {r.status_code}")

        if r.status_code == 200:
            r.raw.decode_content = True

            # Crée le répertoire s'il n'existe pas
            dir_path.mkdir(parents=True, exist_ok=True)

            # Nom de fichier à partir de l’URL (par exemple : dernier segment)
            file_name = Path(image_url).name or "image.jpg"
            image_path = dir_path / file_name

            # Écriture du fichier avec la méthode Path.open()
            with image_path.open('wb') as image_file:
                image_file.write(r.content)

            time.sleep(request_pause)
            print(f"Image enregistrée : {image_path}")
        else:
            print(f"Échec du téléchargement. Code de statut : {r.status_code}")

    except requests.exceptions.SSLError as e:
        print(f"Erreur SSL : {e}")
    except Exception as e:
        print(f"Erreur : {e}")


# Exemple d'utilisation
image_url = 'https://www.e-codices.unifr.ch/loris/bbb/bbb-0318/bbb-0318_008r.jp2/full/full/0/default.jpg'
dir_path = Path('/Users/marioncharpier/Documents/Cours_ENC/IIIF_Images')
request_pause = 0

download_image_from_url(image_url, dir_path, request_pause)
